In [1]:
import requests
import pandas as pd
import numpy as np
import time

api_key =  'RGAPI-4b716c74-5faf-4cff-88a4-617f81317dc5'
sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +'hide on bush' +'?api_key=' + api_key
r = requests.get(sohwan)
r

<Response [200]>

In [2]:
grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(grandmaster)#그마데이터 호출
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()
league_df.to_csv('챌린저데이터.csv',index=False,encoding = 'cp949')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          300 non-null    object
 1   summonerId    300 non-null    object
 2   summonerName  300 non-null    object
 3   leaguePoints  300 non-null    object
 4   wins          300 non-null    object
 5   losses        300 non-null    object
 6   veteran       300 non-null    object
 7   inactive      300 non-null    object
 8   freshBlood    300 non-null    object
 9   hotStreak     300 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB


In [9]:
#league_df = pd.read_csv('그마데이터.csv', encoding='cp949')

In [3]:
league_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,gMbHnjgHzfVyfBLMHAuRblf_W7jYgxS8nl7B5OBRLW5F5hs,다없수질,1042,255,211,False,False,True,True
1,CHALLENGER,-7F8rXtKir7tfOVVmmHej7mAM8B-tIs2tB2nDQPtUfnS1yg,기매강,753,403,376,False,False,True,False
2,CHALLENGER,QGoXjcLzEjQf8bvJQbKMk_Fixl_EiLSozm2aFJQdAgqBJAc,Jad1,866,369,318,False,False,False,False
3,CHALLENGER,sl0easyaZt3ZcO42Akq6yCUzqfvwXHhFr2auYligDvE-7BM,Gen G Moo,779,417,374,True,False,False,False
4,CHALLENGER,ACNSvZuwSrWvt9uvUQ_vZr23MyOZJzXIvVzt6hTU3ZzMjEo,원딜 왕자 이채환,1013,183,134,False,False,False,False


In [4]:
league_df['account_id'] = np.nan

In [5]:
for i in range(len(league_df)):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(10)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
            
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
    
    except:
        pass

In [6]:
league_df.isnull().sum()

tier            0
summonerId      0
summonerName    0
leaguePoints    0
wins            0
losses          0
veteran         0
inactive        0
freshBlood      0
hotStreak       0
account_id      2
dtype: int64

In [7]:
league_df.to_csv('챌린저데이터_accountid.csv',index=False,encoding = 'cp949')

In [8]:
league_df2 = league_df[league_df['account_id'].notnull()]

In [9]:
league_df2.shape

(298, 11)

In [10]:
league_df2.head(10)

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,account_id
0,CHALLENGER,gMbHnjgHzfVyfBLMHAuRblf_W7jYgxS8nl7B5OBRLW5F5hs,다없수질,1042,255,211,False,False,True,True,eMiil_F4MgoZVSnrDXWuAcq4oHvPLjfLd3BKlt1z4K_dT_E
1,CHALLENGER,-7F8rXtKir7tfOVVmmHej7mAM8B-tIs2tB2nDQPtUfnS1yg,기매강,753,403,376,False,False,True,False,VpuMvqVd66J945tp0Tjz2pFT2kNph8Z0CrmI2d40qBnHxq...
2,CHALLENGER,QGoXjcLzEjQf8bvJQbKMk_Fixl_EiLSozm2aFJQdAgqBJAc,Jad1,866,369,318,False,False,False,False,ZF9qugOUJHGi_qJOfcBWrl6WX5W-eGoB16OULRhPD-kYRM...
3,CHALLENGER,sl0easyaZt3ZcO42Akq6yCUzqfvwXHhFr2auYligDvE-7BM,Gen G Moo,779,417,374,True,False,False,False,HeCKqMudR4PcoDNXMugI720pH6c1V8AWbPL2DpZBLYuo0y...
4,CHALLENGER,ACNSvZuwSrWvt9uvUQ_vZr23MyOZJzXIvVzt6hTU3ZzMjEo,원딜 왕자 이채환,1013,183,134,False,False,False,False,2WTeAUFzqchiWqEarTFMq41WBPXEe1AyPqrgNn8aPjePIPA
5,CHALLENGER,W8cDq7hA-3ijNhXx0dyaYIaD6aZMXoB2wPMO3-pveVzGSBY,hearer,768,86,50,False,False,False,False,76drUDVwqykTvgQT91Wwv2o5Cdhj_GDhgNa53FG24QQmltw
6,CHALLENGER,gtSWtH4UeA6nNuXVpncBfmauzk_zV70xcEcMTKfHMZPEia0,FFFFKKKK,796,191,159,True,False,False,False,12FL5Bt7e7NQmMo0_f9ezQIDqGpi3IJfg2PK3BTIuPtOPt...
7,CHALLENGER,VNunHT3PNC9fdM9I7RMTanWlPZ44h1jMeqWv_zUnSMQwFG0,너와나의연결Gori,1007,313,274,True,False,False,False,EMIDFTl5K7XjMQrc_FK4uML_iT2OAq11vC74RgwPPsXCdK4
8,CHALLENGER,8ip6KT-zJFBqsv6jKbwysU2jJNZUet8rnWs3aeImsLAcZ0c,DRX Deft,1173,340,294,True,False,False,True,JUwr-8WxqdRx_bX9jPQgiUu5i3dxycpODfbOZx3zELjTHbc
9,CHALLENGER,sB4eB7RvgpaI3VlhOVkxKoP9N5iOd4UUX1-bRch0WoQsCGg,happy game,937,710,625,True,False,False,False,dDQbn5lOfi1aKwjno8VvxMcJqLaTCaFXK7wOgV94KYDHxzQ


In [11]:
match_info_df = pd.DataFrame()
season = str(13)
for i in range(len(league_df2)):
    try:
        match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df2['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match0)
        
        while r.status_code == 429:
            time.sleep(5)
            match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df2['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)
            
        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    
    except Exception as e:
        print(i)
        print(e)

149
'matches'


In [12]:
match_info_df.head()

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4424184099,223,420,13,1591116239501,DUO_SUPPORT,BOTTOM
1,KR,4424078565,412,420,13,1591113402698,DUO_SUPPORT,MID
2,KR,4424042071,432,420,13,1591110595447,DUO_SUPPORT,BOTTOM
3,KR,4423937146,111,420,13,1591108906562,DUO_SUPPORT,MID
4,KR,4423747308,111,420,13,1591106008999,DUO_SUPPORT,BOTTOM


In [13]:
match_info_df.isnull().sum()

platformId    0
gameId        0
champion      0
queue         0
season        0
timestamp     0
role          0
lane          0
dtype: int64

In [16]:
match_info_df.shape

(29700, 8)

In [15]:
#match_info_df.to_csv('match_info(챌린저데이터).csv',index=False,encoding = 'cp949')

In [3]:
match_info_df = pd.read_csv('match_info(챌린저데이터).csv', encoding='cp949')

In [6]:
match_fin = pd.DataFrame()



for i in range(16601, len(match_info_df)):    
    
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_info_df['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass

    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        print('loop location : ',i)
        start_time = time.time()

        while True: # 429error가 끝날 때까지 무한 루프
            if r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: #다시 response 200이면 loop escape
                print('total wait time : ', time.time() - start_time)
                print('recovery api cost')
                break

    elif r.status_code == 503: # 잠시 서비스를 이용하지 못하는 에러
        print('service available error')
        start_time = time.time()

        while True:
            if r.status_code == 503 or r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: # 똑같이 response가 정상이면 loop escape
                print('total error wait time : ', time.time() - start_time)
                print('recovery api cost')
                break
    elif r.status_code == 403: # api갱신이 필요
        print('you need api renewal')
        print('break')
        break

    # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성
    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match_fin = pd.concat([match_fin,mat])
    
    
    if i%100 == 0:
        match_fin.to_csv('챌린저데이터(kr_match_fin)tmp4.csv',index=False,encoding = 'cp949')
        f = open('last_saved_index(challenger).txt', 'a')
        data = '\nlast saved index in loop  : {}'.format(i)
        f.write(data)
        f.close()
    

api cost full : infinite loop start
loop location :  16780
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
200
total wait time :  92.67836141586304
recovery api cost
api cost full : infinite loop start
loop location :  16880
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
200
total wait time :  92.67541694641113
recovery api cost
api cost full : infinite loop start
loop location :  16980
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second wait time
429
try 10 second

In [7]:
match_fin.to_csv('챌린저데이터(kr_match_fin)TMP4.csv',index=False,encoding = 'cp949')
match_fin.drop(columns='status', inplace=True)

In [8]:
match_fin.shape

(13121, 13)

In [9]:
data1 = pd.read_csv('챌린저데이터(kr_match_fin)tmp1.csv', encoding='cp949')
data2 = pd.read_csv('챌린저데이터(kr_match_fin)tmp2.csv', encoding='cp949')
data3 = pd.read_csv('챌린저데이터(kr_match_fin)tmp3.csv', encoding='cp949')

In [18]:
data2.shape

(1000, 13)

In [26]:
data1.dropna(inplace=True)
data2.dropna(inplace=True)
data3.dropna(inplace=True)

In [30]:
from ast import literal_eval

data1['participants'] = data1['participants'].apply(literal_eval)
data1['teams'] = data1['teams'].apply(literal_eval)

data2['participants'] = data2['participants'].apply(literal_eval)
data2['teams'] = data2['teams'].apply(literal_eval)

data3['participants'] = data3['participants'].apply(literal_eval)
data3['teams'] = data3['teams'].apply(literal_eval)

In [33]:
challenger_tot = pd.concat([data1, match_fin])
challenger_tot.drop_duplicates(subset='gameId', inplace=True)
challenger_tot.to_csv('match_fin(챌린저데이터).csv', index=False, encoding='cp949')

In [41]:
challenger_tot.isnull().sum()

gameId                   1
platformId               1
gameCreation             1
gameDuration             1
queueId                  1
mapId                    1
seasonId                 1
gameVersion              1
gameMode                 1
gameType                 1
teams                    1
participants             1
participantIdentities    1
dtype: int64

In [38]:
challenger_tot.head()

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4.42418e+09,KR,1.59112e+12,2169,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,4.42408e+09,KR,1.59111e+12,2086,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,4.42404e+09,KR,1.59111e+12,2020,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,4.42394e+09,KR,1.59111e+12,1337,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4.42375e+09,KR,1.59111e+12,1798,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [40]:
challenger_tot.to_csv('challenger_tot(match_fin).csv', index=False, encoding='cp949')

In [42]:
match_fin = challenger_tot

In [2]:
match_fin = pd.read_csv('challenger_tot(match_fin).csv', encoding='cp949')

- load시 array가 string으로 읽히는 이슈 존재함. literal_eval로 해결 가능. nan값이 없어야 함.

In [6]:
from ast import literal_eval

match_fin.dropna(inplace=True)
match_fin['participants'] = match_fin['participants'].apply(literal_eval)
match_fin['teams'] = match_fin['teams'].apply(literal_eval)

In [7]:
match_fin.index = np.arange(match_fin.shape[0])

In [9]:
match_fin.shape

(12858, 13)

## 최종 데이터 만드는 pipeline

In [11]:
stat_df = pd.DataFrame()
key_error_ls = []



for j in range(match_fin.shape[0]):
    blue_ply_df = pd.DataFrame()
    red_ply_df = pd.DataFrame()
    
    
    try : 
        for i in range(10):
            if i<=5.5 :
                one_ply = pd.DataFrame(pd.DataFrame(match_fin.participants.values[j]).stats[i], index=[i])
                blue_ply_df = pd.concat([blue_ply_df, one_ply])
            else :
                one_ply = pd.DataFrame(pd.DataFrame(match_fin.participants.values[j]).stats[i], index=[i])
                red_ply_df = pd.concat([red_ply_df, one_ply])
    
    except KeyError :
        key_error_ls.append(j)
        continue
    
    blue_ply_df.drop(['item0', 'item1','item2', 'item3', 'item4', 'item5', 'item6'], axis=1, inplace=True)
    red_ply_df.drop(['item0', 'item1','item2', 'item3', 'item4', 'item5', 'item6'], axis=1, inplace=True)
    
    blue_tot_stat = pd.DataFrame(blue_ply_df.apply(sum)).T
    tmp_arr = np.array(blue_tot_stat.columns) +'_blue'
    blue_tot_stat.columns = tmp_arr.tolist()
    
    red_tot_stat = pd.DataFrame(red_ply_df.apply(sum)).T
    tmp_arr2 = np.array(red_tot_stat.columns) +'_red'
    red_tot_stat.columns = tmp_arr2.tolist()
    
    
    blue_team_stat = pd.DataFrame(pd.DataFrame(match_fin.teams.values[j]).iloc[0]).T
    blue_stat_index = blue_team_stat.columns.tolist()
    blue_team_stat.columns = [x +'_blue'for x in blue_stat_index]

    red_team_stat = pd.DataFrame(pd.DataFrame(match_fin.teams.values[j]).iloc[1]).T
    #red_team_stat.reset_index(inplace=True)
    red_stat_index = red_team_stat.columns.tolist()
    red_team_stat.columns = [x +'_red'for x in red_stat_index]
    
    team_tot_stat = pd.concat([blue_team_stat, red_team_stat.reset_index(drop=True)], axis=1)
    ply_tot_stat = pd.concat([blue_tot_stat, red_tot_stat.reset_index(drop=True)], axis=1)
        
    all_stat = pd.concat([team_tot_stat, ply_tot_stat], axis=1)
    all_stat.index = [j]
    #tmp_dict[j] = all_stat
    
   
    #if len(tmp_ls)>=1 :
        #try :
            #stat_df.drop(tmp_ls, axis=1, inplace=True)
                  
        #except:
            #pass
    
    
    try : 
        stat_df = pd.concat([stat_df, all_stat])
        
    except :
        pass
        #print('fail to concat: the number of row in match_fin is ',j, 'problem is ',tmp_ls)
    
    

In [12]:
len(key_error_ls)

1

In [13]:
key_error_ls

[7482]

In [26]:
stat_df.loc[7481:7485]

,teamId_blue,win_blue,firstBlood_blue,firstTower_blue,firstInhibitor_blue,firstBaron_blue,firstDragon_blue,firstRiftHerald_blue,towerKills_blue,inhibitorKills_blue,...,perk4Var3_red,perk5_red,perk5Var1_red,perk5Var2_red,perk5Var3_red,perkPrimaryStyle_red,perkSubStyle_red,statPerk0_red,statPerk1_red,statPerk2_red
7481,100,Fail,False,True,False,False,False,False,5,0,...,0,32619,18015,4,0,32300,32500,20026,20032,20008
7483,100,Win,True,True,True,False,True,False,8,2,...,0,33409,1615,0,0,32500,33200,20032,20032,20008


In [30]:
stat_df.dropna().shape

(8522, 226)

In [35]:
stat_df.to_csv('챌린저데이터(13_KR).csv', index=False)

In [20]:
stat_df.head()

,teamId_blue,win_blue,firstBlood_blue,firstTower_blue,firstInhibitor_blue,firstBaron_blue,firstDragon_blue,firstRiftHerald_blue,towerKills_blue,inhibitorKills_blue,...,perk4Var3_red,perk5_red,perk5Var1_red,perk5Var2_red,perk5Var3_red,perkPrimaryStyle_red,perkSubStyle_red,statPerk0_red,statPerk1_red,statPerk2_red
0,100,Win,True,False,True,True,True,False,9,1,...,0,32817,2198,20,0,32300,32700,20025,20026,20010
1,100,Win,False,True,True,True,True,True,10,2,...,0,33822,2464,180,0,32600,32600,20023,20032,20009
2,100,Win,False,True,True,True,True,False,8,1,...,5,33033,48,5,0,32200,32900,20023,20032,20009
4,100,Fail,False,False,False,True,False,True,3,0,...,0,33382,9,0,0,32700,33200,20028,20027,20010
5,100,Win,False,False,True,True,False,False,10,1,...,5,33314,22,0,0,32500,33100,20029,20032,20009


In [36]:
challenger_data = match_fin.join(stat_df, how='inner')

In [37]:
challenger_data.shape

(8524, 239)

In [61]:
((challenger_data['inhibitorKills_red'].iloc[:,0] == challenger_data['inhibitorKills_red'].iloc[:,1])&(challenger_data['inhibitorKills_blue'].iloc[:,0] == challenger_data['inhibitorKills_blue'].iloc[:,1])).sum()

6837

In [52]:
(challenger_data['inhibitorKills_blue'].iloc[:,0] == challenger_data['inhibitorKills_blue'].iloc[:,1]).sum()

7009

In [62]:
(challenger_data['inhibitorKills_red'].iloc[:,0] == challenger_data['inhibitorKills_red'].iloc[:,1]).sum()

7044

In [55]:
challenger_data['inhibitorKills_blue'].iloc[:,0].value_counts(normalize=True)

0    0.445917
1    0.323088
2    0.162717
3    0.053261
4    0.011732
5    0.002464
6    0.000704
8    0.000117
Name: inhibitorKills_blue, dtype: float64

In [82]:
(challenger_data2.firstBlood_blue == challenger_data2.firstBloodKill_blue).sum()

7658

In [83]:
(challenger_data2.firstBlood_red == challenger_data2.firstBloodKill_red).sum()

7658

In [81]:
challenger_data2.shape

(8524, 235)

In [64]:
challenger_data2 = challenger_data.loc[:,~challenger_data.columns.duplicated(keep='first')]

In [63]:
master_grand_data = pd.read_csv('마스터_그마_데이터(13_KR).csv')

In [65]:
master_grand_data.shape

(15575, 235)

In [66]:
challenger_data2.shape

(8524, 235)

In [67]:
set(master_grand_data.columns).difference(challenger_data2.columns)

set()

In [68]:
set(challenger_data2.columns).difference(master_grand_data.columns)

set()

In [69]:
master_grand_data[challenger_data2.columns].shape

(15575, 235)

In [71]:
challenger_data2[master_grand_data.columns].shape

(8524, 235)

In [72]:
cat = pd.concat([master_grand_data, challenger_data2])

In [73]:
cat.shape

(24099, 235)

In [75]:
cat.drop_duplicates(subset='gameId', inplace=True)
cat.dropna(inplace=True)
cat.shape

(22228, 235)

In [79]:
(cat.firstBlood_red == cat.firstBloodKill_red).sum()

20123

In [84]:
drop_ls = ['win_red',
 'firstBlood_red',
 'firstTower_red',
 'firstInhibitor_red',
 'firstBaron_red',
 'firstDragon_red',
 'firstRiftHerald_red',
'firstBloodKill_red',
 'firstBloodAssist_red',
 'firstTowerKill_red',
 'firstTowerAssist_red',
 'firstInhibitorKill_red',
 'firstInhibitorAssist_red']

In [86]:
cat.drop(columns=drop_ls, inplace=True)

In [87]:
cat.shape

(22228, 222)

In [88]:
cat.to_csv('챌린저_마스터_그마데이터(13_KR).csv', index=False)

In [93]:
col_df = pd.DataFrame(cat.columns, columns = ['column_names'])
col_df.to_csv('all_columns.csv', index=False)